<a href="https://colab.research.google.com/github/philip-morlier/big_data_challenge/blob/master/big_data_level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").getOrCreate()

In [18]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
# url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [20]:
review_df = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_df.show(3)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809|2015-08-31 00:00:00|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175|2015-08-31 00:00:00|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668|2015-08-31 00:00:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 3 rows



In [23]:
product_df = df.select(["product_id","product_title"])
product_df.show(3)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
+----------+--------------------+
only showing top 3 rows



In [25]:
customer_df = df.groupby("customer_id").agg({"customer_id":"count"})
customer_df.show(3)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   48670265|                 1|
|   49103216|                 2|
|    1131200|                 1|
+-----------+------------------+
only showing top 3 rows



In [26]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show(3)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 3 rows



In [27]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
# url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-08-31 00:00:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [28]:
review_df = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_df.show(3)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370|2015-08-31 00:00:00|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722|2015-08-31 00:00:00|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390|2015-08-31 00:00:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 3 rows



In [29]:
product_df = df.select(["product_id","product_title"])
product_df.show(3)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00FALQ1ZC|Invicta Women's 1...|
|B00D3RGO20|Kenneth Cole New ...|
|B00DKYC7TK|Ritche 22mm Black...|
+----------+--------------------+
only showing top 3 rows



In [30]:
customer_df = df.groupby("customer_id").agg({"customer_id":"count"})
customer_df.show(3)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|    1567510|                 1|
|   19502021|                 1|
|   12819130|                 1|
+-----------+------------------+
only showing top 3 rows



In [31]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show(3)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 3 rows

